In [1]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import csv
from pprint import pprint

# Google developer API key
from config import gkey

In [2]:
Zip_code = [33114, 33002, 33018, 33017, 33016, 33015, 33014, 33013, 33012, 33011, 33010, 33092, 
         33090, 33039, 33035, 33034, 33031, 33032, 33030, 33033, 33149, 33181, 33182, 33183, 
         33184, 33185, 33186, 33187, 33188, 33189, 33190, 33180, 33179, 33178, 33166, 33167, 
         33168, 33169, 33172, 33173, 33174, 33175, 33176, 33177, 33165, 33193, 33194, 33247, 
         33255, 33256, 33257, 33261, 33265, 33266, 33269, 33280, 33283, 33245, 33243, 33242, 
         33196, 33197, 33199, 33206, 33222, 33231, 33233, 33234, 33238, 33299, 33296, 33164, 
         33101, 33130, 33131, 33132, 33133, 33134, 33135, 33136, 33137, 33138, 33129, 33128, 
         33127, 33102, 33106, 33111, 33112, 33116, 33122, 33124, 33125, 33126, 33170, 33156, 
         33158, 33157, 33161, 33155, 33162, 33153, 33152, 33151, 33150, 33163, 33142, 33143, 
         33144, 33145, 33146, 33147, 33159, 33109, 33139, 33141, 33154, 33140, 33119, 33239, 
         33056, 33160, 33055, 33054]

In [18]:
# set up lists to hold reponse info
lati = []
long = []

In [7]:
# 1. What are the geocoordinates (latitude and longitude) of Zip Codes

target_zip = "33114"

params = {"address": target_zip, "key": gkey}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

print(f"Drill #1: The Geocoordinates of zip code {target_zip}")

# Run request
response = requests.get(base_url, params=params)

# Convert to JSON
zip_geo = response.json()

# Extract lat/lng
lat = zip_geo["results"][0]["geometry"]["location"]["lat"]
lng = zip_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print(f"{target_zip}: {lat}, {lng}")


Drill #1: The Geocoordinates of zip code 33114
33114: 25.7897333, -80.2259474


In [19]:
# Loop through the list of cities and perform a request for data on each
for zipcodes in Zip_code[:10]:
    target_zip = zipcodes
    params = {"address": str(target_zip), "key": gkey}
    response = requests.get(base_url, params=params)
    zip_geo = response.json()
    lati.append(zip_geo["results"][0]["geometry"]["location"]["lat"])
    long.append(zip_geo["results"][0]["geometry"]["location"]["lng"])

In [20]:
lati

[25.7897333,
 25.8665814,
 25.9231399,
 25.9410503,
 25.8925211,
 25.9358702,
 25.9109586,
 25.8618743,
 25.8641207,
 25.824762]

In [25]:
# create a data frame from cities, lat, and temp
zipcode_dict = {
    "zip": Zip_code[:10],
    "latit": lati,
    "longi": long
}
zipcode_data = pd.DataFrame(zipcode_dict)
zipcode_data

,zip,latit,longi
0,33114,25.789733,-80.225947
1,33002,25.866581,-80.304334
2,33018,25.923140,-80.399265
3,33017,25.941050,-80.300777
4,33016,25.892521,-80.335236
5,33015,25.935870,-80.326881
6,33014,25.910959,-80.304597
7,33013,25.861874,-80.271159
8,33012,25.864121,-80.304597
9,33011,25.824762,-80.279327


In [45]:
# Add columns for gym, address, airport rating
# Note that we used "" to specify initial entry.
zipcode_data["Name"] = ""
zipcode_data["Vicinity"] = ""
zipcode_data.head()

,zip,latit,longi,Name,Vicinity
0,33114,25.789733,-80.225947,,
1,33002,25.866581,-80.304334,,
2,33018,25.923140,-80.399265,,
3,33017,25.941050,-80.300777,,
4,33016,25.892521,-80.335236,,


In [67]:
# params dictionary to update each iteration
params2 = {
    "radius": 2000,
    "types": "gym",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in zipcode_data.iterrows():
    # get lat, lng from df
    lat = row["latit"]
    lng = row["longi"]

    # change location each iteration while leaving original params in place
    params2["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    gym_address = requests.get(base_url, params=params2)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    gym_address = gym_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        zipcode_data.loc[index, "Name"] = gym_address["results"][0]["name"]
        zipcode_data.loc[index, "Vicinity"] = gym_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [68]:
zipcode_data.head(20)

,zip,latit,longi,Name,Vicinity
0,33114,25.789733,-80.225947,Landa-Galindez Amalia M MD,"1611 Northwest 12th Avenue # 233, Miami"
1,33002,25.866581,-80.304334,LA Fitness,"630 West 49th Street, Hialeah"
2,33018,25.923140,-80.399265,Legacy Gym,Hialeah Gardens
3,33017,25.941050,-80.300777,Planet Fitness,"18620 Northwest 67th Avenue, Miami"
4,33016,25.892521,-80.335236,Cheer Revolution All Stars,"2500 West 84th Street, Hialeah"
5,33015,25.935870,-80.326881,Planet Fitness,"18620 Northwest 67th Avenue, Miami"
6,33014,25.910959,-80.304597,Shula's Athletic Club,"15150 Bull Run Road, Miami Lakes"
7,33013,25.861874,-80.271159,Парикиахер,Hialeah
8,33012,25.864121,-80.304597,LA Fitness,"630 West 49th Street, Hialeah"
9,33011,25.824762,-80.279327,Hialeah School Of Self Defense,"1982 East 4th Avenue, Hialeah"


In [44]:
# 3. Find the name and address of a gym in the zips. Needs to be gym as it is the type name provided by google
#    Hint: See https://developers.google.com/places/web-service/supported_types
target_type = "gym"
zip_coords = "25.89,-80.225"
radius = 5000

# rewrite params dict

params2 = {
    "location": zip_coords,
    "types": target_type,
    "radius": radius,
    "key": gkey
}

# Build URL using the Google Maps API
base_url2 = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print("Fitness Center near ZIP")

# Run request
response = requests.get(base_url2, params2)

gym_zip = response.json()

# Print the JSON (pretty printed)
print(json.dumps(fitness_zip, indent=4, sort_keys=True))

# Print the name and address of the first bike shop to appear
#print(gym_zip["results"][0]["name"])
#print(gym_zip["results"][0]["vicinity"])

Fitness Center near ZIP
{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 25.8737794,
                    "lng": -80.2424356
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.8751260302915,
                        "lng": -80.24117386970848
                    },
                    "southwest": {
                        "lat": 25.8724280697085,
                        "lng": -80.2438718302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png",
            "id": "cddbbbad8a27647606f44d6cb89b8c4a02ce4238",
            "name": "The Den Fitness",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 4000,
                    "html_attributi

In [24]:
# 6. Bonus: Find the names and addresses of the top five restaurants in your home city.
#    Hint: Read about "Text Search Results"
# (https://developers.google.com/places/web-service/search#TextSearchRequests)
my_phrase = "Fitness Miami"
target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

params = {
    "query": my_phrase,
    "key": gkey
}

print("Fitness centers")

response = requests.get(target_url, params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)
fitness_centers = response.json()
# print(json.dumps(happy_places, indent=4, sort_keys=True))

for place in fitness_centers["results"]:
    print(place["name"])
    print(place["formatted_address"])

Fitness centers
Core Fitness Miami
162 NE 25th St, Miami, FL 33137, USA
LA Fitness
3301 SW 22nd St, Miami, FL 33145, USA
B-Fit Biscayne
5080 Biscayne Blvd, Suite 1-B, Miami, FL 33137, United States
LA Fitness
900 S Miami Ave #236, Miami, FL 33130, USA
Orangetheory Fitness
244 Biscayne Blvd, Miami, FL 33132, USA
Punch Elite Fitness
1901 NW Miami Ct, Miami, FL 33136, USA
Bodytek Fitness Wynwood - Miami
545 NW 28th St, Miami, FL 33127, USA
LegacyFit
77 NE 24th St, Miami, FL 33137, USA
Orangetheory Fitness
9017 Biscayne Blvd, Miami Shores, FL 33138, USA
24 Hour Fitness
2982 Grand Ave, Miami, FL 33133, USA
Orangetheory Fitness
89 NE 38th St, Miami, FL 33137, USA
Equinox South Beach
520 Collins Ave, Miami Beach, FL 33139, USA
ERA Fit
1625 N Miami Ave, Miami, FL 33136, USA
Ironflower Fitness
7205 NE 4th Ave, Miami, FL 33138, USA
Primal Fit 360 Personal Training
9531 NE 2nd Ave, Miami Shores, FL 33138, USA
DBC Fitness
3841 NE 2nd Ave #103, Miami, FL 33137, USA
Society Fitness
Riverfront, 92 SW